In [1]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/node-master'       # path to your project on Drive

GIT_USERNAME = "sgsyang" # replace with yours
#GIT_TOKEN = "XXX"           # definitely replace with yours
#GIT_REPOSITORY = "yyy"      # ...nah


drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
GIT_PATH = "https://github.com/sgsyang/node.git"

!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/node-master’: File exists
Cloning into 'node'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 0), reused 19 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
README.md
          1,555 100%    0.00k

In [4]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1
import os, sys
import time
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
!pip install catboost
!pip install category_encoders
!pip install tensorboardX
# !pip install qhoptim



import lib
import torch, torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

experiment_name = 'Catboost'
experiment_name = '{}_{}.{:0>2d}.{:0>2d}_{:0>2d}:{:0>2d}'.format(experiment_name, *time.gmtime()[:5])
print("experiment:", experiment_name)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_VISIBLE_DEVICES=0,1
     |████████████████████████████████| 204kB 3.2MB/s 
experiment: Catboost_2020.05.28_00:02


In [5]:
torch.cuda.get_device_name(device)

'Tesla P100-PCIE-16GB'

##Default

In [0]:
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

# 13, 1, 602, 295, 970, 2000, 3000, 10000
random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]
def Test(random_state):
  data = lib.Dataset("YAHOO", random_state=random_state, quantile_transform=True, quantile_noise=1e-3)
  in_features = data.X_train.shape[1]

  mu, std = data.y_train.mean(), data.y_train.std()
  normalize = lambda x: ((x - mu) / std).astype(np.float32)
  data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])

  model = CatBoostRegressor(objective = 'RMSE',task_type="GPU",
                           devices='0:1')
  model.fit(data.X_train,data.y_train)

  # valid_preds = model.predict(data.X_valid)
  test_preds = model.predict(data.X_test)
  test_mse = mean_squared_error(data.y_test, test_preds)
  # print('Catboost for Year Prediction')
  # print("MSE of Validation: %f" % (valid_mse))
  # print("MSE of Test: %f" % (test_mse))

  score = test_mse * std ** 2
  print('-----')
  print('The random_state:',random_state)
  print('The Score',score)
  print(f'Test{random_state} is Over!!!')
  return score, random_state


score_results = [] 
key_results = []


for i in random_state:
  score, random_state = Test(i)
  score_results.append(score)
  key_results.append(random_state)




100%|██████████| 337634490/337634490 [00:34<00:00, 9805696.86it/s]


100%|██████████| 50718132/50718132 [00:08<00:00, 6176375.19it/s]


100%|██████████| 118491838/118491838 [00:07<00:00, 15325500.86it/s]


流式输出内容被截断，只能显示最后 5000 行内容。
25:	learn: 0.8099794	total: 1.39s	remaining: 51.9s
26:	learn: 0.8091581	total: 1.43s	remaining: 51.5s
27:	learn: 0.8085134	total: 1.47s	remaining: 51.2s
28:	learn: 0.8080377	total: 1.51s	remaining: 50.6s
29:	learn: 0.8073999	total: 1.56s	remaining: 50.3s
30:	learn: 0.8068498	total: 1.6s	remaining: 50.1s
31:	learn: 0.8063189	total: 1.65s	remaining: 49.8s
32:	learn: 0.8057069	total: 1.71s	remaining: 50s
33:	learn: 0.8053303	total: 1.75s	remaining: 49.7s
34:	learn: 0.8049115	total: 1.79s	remaining: 49.3s
35:	learn: 0.8043825	total: 1.83s	remaining: 48.9s
36:	learn: 0.8037224	total: 1.88s	remaining: 48.8s
37:	learn: 0.8033323	total: 1.92s	remaining: 48.6s
38:	learn: 0.8029696	total: 1.96s	remaining: 48.4s
39:	learn: 0.8025857	total: 2s	remaining: 48.1s
40:	learn: 0.8022476	total: 2.05s	remaining: 48s
41:	learn: 0.8018118	total: 2.09s	remaining: 47.7s
42:	learn: 0.8015217	total: 2.13s	remaining: 47.4s
43:	learn: 0.8010252	total: 2.17s	remaining: 47.2s
44:	learn: 0

In [0]:
import pprint
from statistics import mean 

Result= dict(zip(key_results,score_results))

pprint.pprint(Result, width=1)


mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

{1: 0.570493521308084,
 13: 0.5708087345881557,
 133: 0.5703873785074465,
 295: 0.5698020811810269,
 602: 0.5700756400671233,
 970: 0.5702986946850953,
 1337: 0.5698540546226871,
 2000: 0.5702912297970216,
 3000: 0.5700712135663754,
 10000: 0.5706918671307404}


In [0]:
# random_state = 1337, 133, 13, 1, 602, 295, 970, 2000, 3000, 10000

# random_state = 13

# data = lib.Dataset("YEAR", random_state=random_state, quantile_transform=True, quantile_noise=1e-3)
# in_features = data.X_train.shape[1]

# mu, std = data.y_train.mean(), data.y_train.std()
# normalize = lambda x: ((x - mu) / std).astype(np.float32)
# data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])

# print("mean = %.5f, std = %.5f" % (mu, std))



In [0]:

# from sklearn.metrics import mean_squared_error
# from catboost import CatBoostRegressor
# model = CatBoostRegressor(objective = 'RMSE',task_type="GPU",
#                            devices='0:1')
# model.fit(data.X_train,data.y_train)

# valid_preds = model.predict(data.X_valid)

# valid_mse = mean_squared_error(data.y_valid, valid_preds)
# print("MSE of Validation: %f" % (valid_mse))


In [0]:

# test_preds = model.predict(data.X_test)


# test_mse = mean_squared_error(data.y_test, test_preds)
# print('Catboost for Year Prediction')
# print("MSE of Validation: %f" % (valid_mse))
# print("MSE of Test: %f" % (test_mse))

# print('The random_state:',random_state)
# print('The Score',test_mse * std ** 2)

##Tuned Parameters

In [6]:
!pip install hyperopt
!pip install catboost

import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor


space = {
    'learning_rate':     hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
    'random_strength':hp.quniform("random_strength", 1,20,1),
    'one_hot_max_size':hp.quniform("one_hot_max_size", 0,25,1),
    'l2_leaf_reg': hp.loguniform('l2_leaf_reg',1 ,10),
    'bagging_temperature':hp.uniform('bagging_temperature', 0, 1),
    'leaf_estimation_iterations':hp.quniform("leaf_estimation_iterations", 1,10,1) 
}

In [7]:
##Automate
def my_tuning(space):
    # reg=CatBoostRegressor(learning_rate=our_space['learning_rate'],
    #                       max_depth = our_space['max_depth'],
    #                       colsample_bylevel = our_space['colsample_bylevel'],
    #                       objective = 'RMSE',task_type="GPU",
    #                       devices='0:1' 
    #                       )
    reg=CatBoostRegressor( learning_rate=space['learning_rate'],
                          random_strength = int(space['random_strength']),
                          one_hot_max_size = int(space['one_hot_max_size']),
                          l2_leaf_reg = space['l2_leaf_reg'],
                          bagging_temperature = space['bagging_temperature'],
                          leaf_estimation_iterations = int(space['leaf_estimation_iterations']),
                          objective = 'RMSE',task_type="GPU",
                          devices='0:1')
        
    reg.fit(data.X_train, data.y_train,verbose = False)

    pred = reg.predict(data.X_test)
    mse= mean_squared_error(data.y_test, pred)
    # print ("test loss:", mse)
    #change the metric if you like
    return {'loss':mse, 'status': STATUS_OK }



score_results = [] 
key_results = []

random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]
for i in random_state:
  data = lib.Dataset("YAHOO", random_state=i, quantile_transform=True, quantile_noise=1e-3)
  in_features = data.X_train.shape[1]
  mu, std = data.y_train.mean(), data.y_train.std()
  normalize = lambda x: ((x - mu) / std).astype(np.float32)
  data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])
  print(f'The dataset with random state {i} is used')

  trials = Trials()
  best = fmin(fn=my_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
  # print('Best parameters',best)
  tuned_loss = trials.best_trial['result']['loss']
  print('Best Loss',tuned_loss)
  final = tuned_loss*std**2
  print('FINAL Error:',final)
  score_results.append(final)
  key_results.append(i)

100%|██████████| 337634490/337634490 [00:09<00:00, 34628668.70it/s]


100%|██████████| 50718132/50718132 [00:01<00:00, 32315983.37it/s]


100%|██████████| 118491838/118491838 [00:04<00:00, 26436259.67it/s]


The dataset with random state 1337 is used
100%|██████████| 50/50 [17:50<00:00, 21.40s/it, best loss: 0.5867697244775892]
Best Loss 0.5867697244775892
FINAL Error: 0.5672243635929394
The dataset with random state 133 is used
100%|██████████| 50/50 [17:07<00:00, 20.56s/it, best loss: 0.5868366906232932]
Best Loss 0.5868366906232932
FINAL Error: 0.5672890990893269
The dataset with random state 13 is used
100%|██████████| 50/50 [17:05<00:00, 20.52s/it, best loss: 0.5864247830260275]
Best Loss 0.5864247830260275
FINAL Error: 0.5668909121772019
The dataset with random state 1 is used
100%|██████████| 50/50 [16:51<00:00, 20.23s/it, best loss: 0.5863841860121293]
Best Loss 0.5863841860121293
FINAL Error: 0.5668516674540821
The dataset with random state 602 is used
100%|██████████| 50/50 [16:53<00:00, 20.26s/it, best loss: 0.5863237596224843]
Best Loss 0.5863237596224843
FINAL Error: 0.5667932538737953
The dataset with random state 295 is used
100%|██████████| 50/50 [16:48<00:00, 20.18s/it, be

In [9]:
print('----')
pprint.pprint(Result, width=1)

mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

----


NameError: ignored

In [3]:
a = {1337 :0.5672243635929394,
133: 0.5672890990893269,
13:  0.5668909121772019,
1: 0.5668516674540821,
602:0.5667932538737953,
295:0.5670528364495863,
970:0.5674049933022387,
2000:   0.5670479367589107,
3000:  0.5669614256553261,
10000:0.5665511013874774
}
score_results = a.values()
mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

Mean: 0.5670067589740886
Max: 0.5674049933022387
Min: 0.5665511013874774
Distance to max 0.0003982343281501155
Distance to min: -0.00045565758661114675
